In [43]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, pointbiserialr
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

In [44]:
# Import the dataset
math = pd.read_csv("./student-mat.csv", sep=';', header=0)
por = pd.read_csv("./student-por.csv", sep=';', header=0)

In [45]:
#correlation of the dependent variables with the independent variables
#########   Correlation with the method from David Boules


# F-statistics for correlation 
#target features is the final grade G3

target = "G3"
# Separate numerical and categorical columns
numerical_features = por.select_dtypes(include=['number']).columns
categorical_features = por.select_dtypes(include=['object', 'category']).columns

X = por.drop(columns = [target])
y = por[target]

# one way ANOVA test for the correlation of means between two or more groups
# assumption the data is normally distributed


from scipy.stats import f_oneway

#correlation between the categorical variables and the target variable of G3(final grading)
from sklearn.feature_selection import f_classif 
for col in categorical_features: 
    groups = [y[X[col] == value] for value in X[col].unique()]
    f_stat, p_value = f_oneway(*groups)
    print(f"Feature: {col}, F_statistic:{f_stat}, P-value: {p_value}")

# filter numerical features
numerical_features= X.select_dtypes(include = [np.number]).columns

#perform F-test

f_scores, p_values = f_classif(X[numerical_features], y)

#display the results

for col, f_score, p_value in zip(numerical_features, f_scores, p_values):
    print(f"Feature: {col}, F_statistic:{f_stat}, P-value: {p_value}")

selected_features_num = [col for col, p_value in zip(numerical_features, p_values) if p_value < 0.05]
selected_features_cat = [col for col, p_value in zip(categorical_features, p_values) if p_value < 0.05]

print("Selected Features:", selected_features_num, selected_features_cat)

Feature: school, F_statistic:56.89067686337133, P-value: 1.5661990923002604e-13
Feature: sex, F_statistic:10.962308407124874, P-value: 0.0009815287061373317
Feature: address, F_statistic:18.707910527412754, P-value: 1.7641534609222437e-05
Feature: famsize, F_statistic:1.3137906447644496, P-value: 0.2521332216658279
Feature: Pstatus, F_statistic:0.0003677569375753126, P-value: 0.984705825951084
Feature: Mjob, F_statistic:7.370224291121831, P-value: 8.30514988494739e-06
Feature: Fjob, F_statistic:3.2726805958419667, P-value: 0.011376280623605892
Feature: reason, F_statistic:10.248465509132915, P-value: 1.3416422874904278e-06
Feature: guardian, F_statistic:2.63816697870449, P-value: 0.07226239503367116
Feature: schoolsup, F_statistic:2.865641360496058, P-value: 0.09097103846579366
Feature: famsup, F_statistic:2.2759054076786045, P-value: 0.1318865120420208
Feature: paid, F_statistic:1.955810225639281, P-value: 0.16244124863569093
Feature: activities, F_statistic:2.321335165021925, P-value

In [46]:
# Encode the categorical variables and scale the numerical variables

from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from sklearn.metrics import r2_score

# Define the target column and features
target_column = "G3"
X = por.drop(columns=[target_column])
X = X[['Medu', 'failures', 'Dalc', 'Walc', 'absences', 'G1', 'G2', 'sex', 'Mjob', 'schoolsup', 'famsup', 'activities', 'nursery', 'higher']]
y = por[target_column]

# Apply the scaler 
numerical_cols = X.select_dtypes(include=['number']).columns
sc_X = StandardScaler()
X[numerical_cols] = sc_X.fit_transform(X[numerical_cols])

#scaling the y is not necessary
#sc_y = StandardScaler()
#y = sc_y.fit_transform(y)

# Encode categorical columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns
for col in categorical_features:
    X[col] = LabelEncoder().fit_transform(X[col])


In [47]:
#splitting the dataset into the training and the test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [48]:
# train the model
from sklearn.svm import SVR
SVR(kernel='rbf')
regressor.fit(X_train, y_train)

SVR()

In [58]:
import numpy as np
from sklearn.metrics import r2_score

# Assuming `regressor` is already trained and `X_test` and `y_test` are defined

# Make predictions using the trained model
y_pred = regressor.predict(X_test)

# Convert y_test to a NumPy array and ensure both y_pred and y_test are reshaped into 2D arrays
y_pred = y_pred.reshape(-1, 1)  # Reshape to (n_samples, 1)

# Convert y_test to NumPy array and reshape
y_test = y_test.to_numpy().reshape(-1, 1)  # Convert y_test to NumPy array and reshape

# Concatenate and print the predictions vs actual values
np.set_printoptions(precision=2, suppress=True)  # Suppress scientific notation
comparison = np.concatenate((y_pred, y_test), axis=1)  # Concatenate along columns (axis=1)
print("Predictions vs Actual:")
print(comparison)

# Evaluating the model performance on the test set
r2_test_score = r2_score(y_test, y_pred)
print(f"R² score on the test set: {r2_test_score}")

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [60]:
## Applying k-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 73.75 %
Standard Deviation: 6.16 %


In [62]:
# Hyperparameter tuning

import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR  # Assuming you're using Support Vector Regression

# Define your regressor (Support Vector Regression in this case)
regressor = SVR()

# Define the parameter grid for hyperparameter tuning
parameters = [
    {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
    {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
]

# Perform GridSearchCV with 10-fold cross-validation
grid_search = GridSearchCV(estimator=regressor,
                           param_grid=parameters,
                           scoring='r2',  # Use R² for regression scoring
                           cv=10,  # 10-fold cross-validation
                           n_jobs=-1)  # Use all available cores
grid_search.fit(X_train, y_train)

# Get the best R² score and the best hyperparameters
best_r2 = grid_search.best_score_
best_parameters = grid_search.best_params_

# Print the best R² score and best parameters
print(f"Best R² score: {best_r2:.2f}")
print("Best Parameters:", best_parameters)

# Optionally, you can refit the model on the entire training data with the best parameters:
best_regressor = grid_search.best_estimator_

# Now you can make predictions and evaluate on the test set:
y_pred = best_regressor.predict(X_test)
# Fit the model with the best hyperparameters found by GridSearchCV
best_regressor = grid_search.best_estimator_
best_regressor.fit(X_train, y_train)

# Predict using the test data
y_pred = best_regressor.predict(X_test)

# Display predictions vs actual values (make sure y_test is in the same shape as y_pred)
# If y_test is a pandas Series, we need to convert it to a numpy array or use its values
y_test = y_test.to_numpy() if isinstance(y_test, pd.Series) else y_test  # Ensure y_test is numpy array

# Concatenate and print the predictions vs actual values
comparison = np.column_stack((y_pred, y_test))
print("Predictions vs Actual:")
print(comparison)

# Evaluating the model performance on the test set
r2_test_score = r2_score(y_test, y_pred)  # Evaluating on the actual test set
print(f"R² score on the test set: {r2_test_score}")

Best R² score: 0.83
Best Parameters: {'C': 1, 'kernel': 'linear'}
Predictions vs Actual:
[[ 7.85  8.  ]
 [14.28 15.  ]
 [15.9  16.  ]
 [10.46 10.  ]
 [ 9.4  10.  ]
 [12.8  12.  ]
 [13.26 13.  ]
 [18.28 17.  ]
 [12.05 12.  ]
 [11.69 12.  ]
 [10.85 11.  ]
 [10.5  10.  ]
 [13.47 13.  ]
 [ 8.84  8.  ]
 [18.04 18.  ]
 [12.28 12.  ]
 [12.91 13.  ]
 [12.45 13.  ]
 [10.67 10.  ]
 [10.19 10.  ]
 [12.28 12.  ]
 [10.56 10.  ]
 [17.19 17.  ]
 [13.17 15.  ]
 [12.53 14.  ]
 [ 2.01  0.  ]
 [12.33 12.  ]
 [13.26 14.  ]
 [11.53 12.  ]
 [12.57  9.  ]
 [13.57 13.  ]
 [16.64 16.  ]
 [13.29 13.  ]
 [16.24 16.  ]
 [12.6  12.  ]
 [ 9.   10.  ]
 [ 9.66 10.  ]
 [11.42 11.  ]
 [12.76 13.  ]
 [11.55 10.  ]
 [15.34 15.  ]
 [17.12 18.  ]
 [11.6  11.  ]
 [13.28 13.  ]
 [12.45 13.  ]
 [ 9.78 10.  ]
 [12.82 14.  ]
 [ 9.53  9.  ]
 [11.59 11.  ]
 [ 9.4  10.  ]
 [ 6.01  8.  ]
 [14.62 17.  ]
 [ 9.38  9.  ]
 [12.25 13.  ]
 [ 7.43  8.  ]
 [10.98 11.  ]
 [11.77 12.  ]
 [11.08 12.  ]
 [14.36 15.  ]
 [14.34 15.  ]
 [13.22 13.